# Make figures

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import glob
import sys
import seaborn as sns
from tqdm.auto import tqdm
from matplotlib.patches import Rectangle
import matplotlib
import rioxarray as rxr
import xarray as xr
from scipy.stats import median_abs_deviation
import warnings
warnings.filterwarnings('ignore')

In [2]:
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/glacier-snow-cover-analysis/'
sys.path.append(os.path.join(base_path, 'functions'))
import utils as f

scm_path = '/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/'
# scm_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/'
figures_out_path = os.path.join(base_path, 'figures')
eras_fn = os.path.join(scm_path, 'analysis', 'all_era_data.csv')
clusters_fn = os.path.join(scm_path, 'analysis', 'climate_clusters.csv')
aois_fn = os.path.join(scm_path, 'analysis', 'all_aois.shp')

## Define some colormaps

In [3]:
# Climate clusters
cluster_cmap_dict = {'W. Aleutians': '#dd3497', 
                     'Maritime': '#018571',
                     'Transitional-Maritime': '#80cdc1',
                     'Transitional-Continental': '#dfc27d',
                     'Continental': '#a6611a',
                     }

## Define order of clusters and subregions for plotting

In [4]:
cluster_order = ['W. Aleutians', 'Maritime', 'Transitional-Maritime', 'Transitional-Continental', 'Continental']
subregion_order = ['N. Rockies', 'Alaska Range', 'W. Chugach Mtns.', 'St. Elias Mtns.', 'N. Coast Ranges',
                   'Aleutians', 'N. Cascades', 'C. Rockies', 'S. Cascades', 'S. Rockies']
# separate subregions for plotting
group1 = ['N. Rockies', 'W. Chugach Mtns.', 'N. Coast Ranges', 'N. Cascades', 'S. Cascades']
group2 = ['Alaska Range', 'St. Elias Mtns.', 'Aleutians', 'C. Rockies', 'S. Rockies']

In [ ]:
# Count total number of observations
rgi_ids = [os.path.basename(x) for x in sorted(glob.glob(os.path.join(scm_path, 'study-sites', 'RGI*')))]
nobs = np.zeros(len(rgi_ids))
for i, rgi_id in enumerate(rgi_ids):
    if rgi_id=='RGI60-01.17761':
        im_classified_fns = glob.glob(os.path.join(scm_path, 'study-sites', 'RGI60-01.17761', 'imagery', 'snowlines', '20*.csv'))
    else:
        im_classified_fns = (glob.glob(os.path.join(scm_path, 'study-sites', rgi_id, 'classified', '20*.nc'))
                             + glob.glob(os.path.join(scm_path, 'study-sites', rgi_id, 'imagery', 'classified', '20*.nc')))
    if len(im_classified_fns) < 1:
        print(rgi_id)
    nobs[i] = len(im_classified_fns)

print(f'Total number of observations = {np.sum(nobs)}')
print(f'Mean observations per site = {np.mean(nobs)}')
print(f'Median observations per site = {np.median(nobs)}')
print(f'Minimum observations per site = {np.min(nobs)}')
print(f'Maximum observations per site = {np.max(nobs)}')


## Figure 1. Study sites and climate clusters

In [ ]:
# Load climate clusters / mean climate
clusters = pd.read_csv(clusters_fn)
print('Climate clusters loaded')

# Load AOIs
aois = gpd.read_file(aois_fn)
# Add climate clusters column
aois = pd.merge(aois, clusters[['RGIId', 'clustName']], on='RGIId')
print('AOIs loaded')

# Load RGI O2 Regions
rgi_O2_fn = os.path.join(scm_path, '..', 'GIS_data', 'RGI', 'RGIv7_02Regions', 
                                'RGI2000-v7.0-o2regions-Alaska-westernCanadaUS_clipped_to_country_outlines.shp')
rgi_O2 = gpd.read_file(rgi_O2_fn)
# remove Brooks Range
rgi_O2 = rgi_O2.loc[rgi_O2['o2region']!='01-01']
# add subregion name and color column
rgi_O2[['Subregion', 'color']] = '', ''
for i, o1o2 in enumerate(rgi_O2['o2region'].drop_duplicates().values):
    o1 = int(o1o2[0:2])
    o2 = int(o1o2[3:])
    subregion_name, color = f.determine_subregion_name_color(o1, o2)
    rgi_O2.loc[rgi_O2['o2region']==o1o2, 'Subregion'] = subregion_name
print('RGI O2 regions loaded')

# Load GTOPO30
gtopo_fn = '/Users/raineyaberle/Research/PhD/GIS_data/GTOPO30_clip.tif'
gtopo = rxr.open_rasterio(gtopo_fn)
gtopo = xr.where(gtopo==-32768, np.nan, gtopo)
print('GTOPO30 loaded')

In [ ]:
# Set up figure
fontsize=12
plt.rcParams.update({'font.size':fontsize, 'font.sans-serif':'Arial'})
fig = plt.figure(figsize=(10,10))
ax = [fig.add_subplot(1,1,1),
      fig.add_axes([0.26, 0.24, 0.27, 0.23]),
      fig.add_axes([0.82, 0.58, 0.07, 0.07])]

### a) Map view
# GTOPO hillshade
ls = matplotlib.colors.LightSource(azdeg=90, altdeg=45)
ax[0].imshow(ls.hillshade(gtopo.data[0], vert_exag=0.002), cmap='gray', alpha=0.5,
             extent=(np.min(gtopo.x.data), np.max(gtopo.x.data), 
                     np.min(gtopo.y.data), np.max(gtopo.y.data)))
# RGI O2 region outlines
color = '#525252'
rgi_O2.plot(ax=ax[0], alpha=1.0, facecolor='None', edgecolor=color, linewidth=1)
ax[0].set_yticks(np.linspace(45, 65, num=6))
ax[0].set_xlim(-167, -112)
ax[0].set_ylim(46, 66)
ax[0].set_xlabel('Longitude ($^{\circ}$E)')
ax[0].set_ylabel('Latitude ($^{\circ}$N)')
ax[0].set_aspect(2.2)
# Site locations
sns.scatterplot(data=aois, x='CenLon', y='CenLat', edgecolor='k', linewidth=0.5, s=20,
                hue='clustName', palette=cluster_cmap_dict, hue_order=cluster_order, alpha=1, ax=ax[0])
handles, labels = ax[0].get_legend_handles_labels()
ax[0].legend().remove()
# Add region labels and arrows
fontweight = 'bold'
background_color = [1, 1, 1, 0.5]
ax[0].text(-160.5, 56.5, f"Aleutians\n(N={len(aois.loc[aois['Subregion']=='Aleutians'])})", ha='center', color=color, rotation=35, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-157, 62.5, f"Alaska Range\n(N={len(aois.loc[aois['Subregion']=='Alaska Range'])})", ha='center', color=color, backgroundcolor=background_color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-147.7, 57.5, f"W. Chugach \nMtns.\n(N={len(aois.loc[aois['Subregion']=='W. Chugach Mtns.'])})", ha='center', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-147.6, 58.9, 0, 0.6, color=color, linewidth=2, head_width=0.34, head_length=0.2)
ax[0].text(-141.5, 57.9, f"St. Elias Mtns.\n(N={len(aois.loc[aois['Subregion']=='St. Elias Mtns.'])})", ha='center', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-141.5, 58.9, 0, 0.6, color=color, linewidth=2, head_width=0.34, head_length=0.2)
ax[0].text(-141, 56, f"N. Coast Ranges\n(N={len(aois.loc[aois['Subregion']=='N. Coast Ranges'])})", ha='center', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-137.1, 56.5, 1.2, 0, color=color, linewidth=2, head_width=0.2, head_length=0.3)
ax[0].text(-133, 50.9, f"N. Cascades\n(N={len(aois.loc[aois['Subregion']=='N. Cascades'])})", ha='center', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-129.9, 51.4, 1.2, 0, color=color, linewidth=2, head_width=0.2, head_length=0.3)
ax[0].text(-129.5, 47, f"S. Cascades\n(N={len(aois.loc[aois['Subregion']=='S. Cascades'])})", ha='center', color=color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].arrow(-126.3, 47.5, 1.2, 0, color=color, linewidth=2, head_width=0.2, head_length=0.3)
ax[0].text(-129.5, 64.5, f"N. Rockies\n(N={len(aois.loc[aois['Subregion']=='N. Rockies'])})", ha='center', color=color, backgroundcolor=background_color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-119, 54.5, f"C. Rockies\n(N={len(aois.loc[aois['Subregion']=='C. Rockies'])})", ha='center', color=color, backgroundcolor=background_color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)
ax[0].text(-115, 47, f"S. Rockies\n(N={len(aois.loc[aois['Subregion']=='S. Rockies'])})", ha='center', color=color, backgroundcolor=background_color, rotation=0, fontsize=fontsize-3, fontweight=fontweight)

### b) Mean weather conditions
sns.scatterplot(data=clusters, x='mean_annual_temp_range', y='mean_annual_precip_cumsum', s=20,
                edgecolor='k', linewidth=0.5, hue='clustName', palette=cluster_cmap_dict, alpha=1, 
                hue_order=cluster_order, legend=False, ax=ax[1])
ax[1].set_xlabel('Air temperature range [$^{\circ}$C]', fontsize=fontsize-2)
ax[1].set_ylabel('Precipitation sum [m]', fontsize=fontsize-2)
ax[1].tick_params(labelsize=fontsize-2)

# Legend
legend = fig.legend(handles, labels, loc='upper right', title='Climate cluster', bbox_to_anchor=[0.7, 0.46, 0.2, 0.2], 
                    fontsize=fontsize-1, markerscale=2, alignment='left', labelspacing=0.6, framealpha=1)

# pie chart inset
pie_labels = cluster_order
pie_sizes = [len(clusters.loc[clusters['clustName']==label]) for label in pie_labels]
ax[2].pie(pie_sizes, colors=list(cluster_cmap_dict.values()), autopct=lambda p: f'{p * sum(pie_sizes) / 100:.0f}', 
        wedgeprops={'linewidth': 0.5, 'edgecolor': 'k'}, textprops={'fontsize': 6, 'fontweight': 'bold', 'color': 'k'},
        pctdistance=0.8)
ax[2].set_zorder(legend.get_zorder() + 1)

# Add text labels
text_labels = ['a', 'b']
for i, axis in enumerate(ax[0:-1]):
    if i==1:
        scale = 0.85
    else:
        scale = 0.93
    axis.text((axis.get_xlim()[1] - axis.get_xlim()[0])*scale + axis.get_xlim()[0],
              (axis.get_ylim()[1] - axis.get_ylim()[0])*scale + axis.get_ylim()[0],
              text_labels[i], fontsize=fontsize+4, fontweight='bold')

# fig.tight_layout()
plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'fig01_study_sites_clusters.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)


## Figure 2. Median AARs and timing comparisons

In [ ]:
cmap = sns.color_palette('mako', n_colors=len(subregion_order)+1)
cmap_dict = dict([[subregion, color] for subregion, color in list(zip(subregion_order, cmap))])

# -----Load glacier boundaries
aois = gpd.read_file(aois_fn)
print('Glacier boundaries loaded')

# -----Load climate clusters
clusters = pd.read_csv(clusters_fn)

# -----Load median AARs for all sites
min_scs_fn = os.path.join(scm_path, 'analysis', 'min_snow_cover_stats.csv')
min_scs = pd.read_csv(min_scs_fn)
# Add difference from September AAR
# min_scs['AAR_P50_diff'] = min_scs['AAR_P50_WOY39'] - min_scs['AAR_P50_min']
# Add Subregion and climate cluster info
min_scs[['CenLon', 'CenLat', 'Subregion', 'clustName']] = 0, 0, '', ''
for rgi_id in min_scs['RGIId'].drop_duplicates().values:
    cenlon, cenlat, subregion = aois.loc[aois['RGIId']==rgi_id, ['CenLon', 'CenLat', 'Subregion']].values[0]
    cluster = clusters.loc[clusters['RGIId']==rgi_id, 'clustName'].values[0]
    min_scs.loc[min_scs['RGIId']==rgi_id, ['CenLon', 'CenLat', 'Subregion', 'clustName']] = cenlon, cenlat, subregion, cluster
# Sort by subregion order
min_scs['Subregion'] = pd.Categorical(min_scs['Subregion'], subregion_order)
min_scs.sort_values(by='Subregion', inplace=True)
print('Median AARs loaded')

# -----Load melt season timings estimate
melt_season_fn = os.path.join(scm_path, 'analysis', 'melt_season_timing.csv')
melt_season = pd.read_csv(melt_season_fn)
# Add subregion column
melt_season['Subregion'] = ''
for rgi_id in melt_season['RGIId'].drop_duplicates().values:
    subregion = aois.loc[aois['RGIId']==rgi_id, 'Subregion'].values[0]
    melt_season.loc[melt_season['RGIId']==rgi_id, 'Subregion'] = subregion
# Sort by subregion order
melt_season['Subregion'] = pd.Categorical(melt_season['Subregion'], subregion_order)
melt_season.sort_values(by='Subregion', inplace=True)
print('Melt season timing loaded')

In [ ]:
fontsize=12
lw = 1.0
plt.rcParams.update({'font.size': fontsize, 'font.sans-serif':'Arial'})
gs = matplotlib.gridspec.GridSpec(10,2, wspace=0.0, hspace=0.0)
fig = plt.figure(figsize=(8,8))
ax = []

# Iterate over subregions
median_color = 'w'
fill_color = '#2171b5'
Iax = -1
for i, subregion in enumerate(min_scs['Subregion'].unique()):
    min_scs_subregion = min_scs.loc[min_scs['Subregion']==subregion]
    
    # a) AARs
    ax.append(fig.add_subplot(gs[i,0]))
    Iax += 1
    min_scs_subregion_melt = pd.melt(
            min_scs_subregion,
            id_vars=['RGIId', 'Subregion'],
            value_vars=['AAR', 'AAR_Sept'],
            var_name='AAR_type',
            value_name='blank'
    )
    min_scs_subregion_melt.rename(columns={'blank': 'AAR'}, inplace=True)
    sns.violinplot(data=min_scs_subregion_melt, x='AAR', hue='AAR_type', cut=0, orient='h', 
                   split=True, inner='quart', fill=True, linewidth=lw, linecolor='k', dodge=True, 
                   width=0.8, palette={'AAR': fill_color, 'AAR_Sept': 'w'}, ax=ax[Iax])
    ax[Iax].set_xlim(0,1)
    ax[Iax].spines[['right', 'top']].set_visible(False)
    if i > 0:
        ax[Iax].spines['top'].set_visible(True)
        ax[Iax].spines['top'].set_color('gray')
    handles, labels = ax[Iax].get_legend_handles_labels()
    ax[Iax].legend().remove()
    if i==0:
        ax[Iax].text(0.8, 0.7, 'a', transform=ax[Iax].transAxes, 
                     fontweight='bold', fontsize=fontsize+4, color='k')
        ax[Iax].set_xlabel('AAR')
        labels = ['Observed AAR', 'September AAR']
        ax[Iax].legend(handles, labels, loc='upper center', ncols=2, frameon=False, bbox_to_anchor=[0.4, 1.6, 0.2, 0.2])
    else:
        ax[Iax].set_ylabel('')
    if i < 9:
        ax[Iax].set_xticks([])
        ax[Iax].set_xlabel('')
        ax[Iax].spines['bottom'].set_color('gray')
    else:
        ax[Iax].set_xlabel('AAR')
    ax[Iax].set_ylabel(subregion, ha='right', va='center', color='k', rotation=0)

    # b) AAR timing and melt season duration
    ax.append(fig.add_subplot(gs[i,1]))
    Iax += 1
    k = sns.kdeplot(min_scs_subregion['WOY'], vertical=False, color=fill_color, 
                    fill=True, edgecolor='k', linewidth=lw, alpha=1, ax=ax[Iax], zorder=2)
    median = min_scs_subregion['WOY'].median()
    ax[Iax].plot([median, median], [0, ax[Iax].get_ylim()[1]*0.9], '-', color='w', linewidth=lw+0.5, zorder=3)
    ax[Iax].plot([14,45], [0,0], '-', color='k', linewidth=2)
    melt_season_subregion = melt_season.loc[melt_season['Subregion']==subregion]
    melt_season_start = melt_season_subregion['melt_season_start_WOY'].mean()
    melt_season_end = melt_season_subregion['melt_season_end_WOY'].mean()
    ax[Iax].fill_between([melt_season_start, melt_season_end],  
                         [0, 0], [ax[Iax].get_ylim()[1], ax[Iax].get_ylim()[1]],
                         facecolor='k', edgecolor='None', alpha=0.15, zorder=1, label='Melt season duration')
    ax[Iax].set_xlim(13,45)
    ax[Iax].set_yticks([])
    ax[Iax].set_ylabel('')
    ax[Iax].set_xticks([])
    ax[Iax].set_xlabel('')
    if i==9:
        ax[Iax].set_xticks([18, 22, 26, 31, 35, 39, 44])
        ax[Iax].set_xticklabels(['May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov'])
        ax[Iax].set_xlabel('Time of year')
    if i==0:
        ax[Iax].legend(loc='upper center', frameon=False, bbox_to_anchor=[0.4, 1.6, 0.2, 0.2])
        ax[Iax].text(0.8, 0.7, 'b', transform=ax[Iax].transAxes, 
                     fontweight='bold', fontsize=fontsize+4, color='k')
    ax[Iax].spines[['left', 'right', 'top', 'bottom']].set_visible(False)
            
plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'fig02_median_aars+timings.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

In [ ]:
# Check WOY and months
# df = pd.DataFrame({'Date': pd.date_range('2013-01-01', '2023-12-30')})
# df['WOY'] = df['Date'].dt.isocalendar().week
# df['year'] = df['Date'].dt.year
# df['month'] = df['Date'].dt.month
# df['day'] = df['Date'].dt.day
# df.loc[(df['month']==10) & (df['day']==1)]['WOY']#.median()

In [ ]:
# Print statistics

print('All sites:')
print('-----------')
print('---AARs---')
print(min_scs['AAR'].describe())
print('\n---WOYs---')
print(min_scs['WOY'].describe()) 

print(' ')
print('By subregion:')
print('-----------')
print('---AARs---')
print(min_scs.groupby('Subregion')['AAR'].describe())

print('\n---WOYs---')
print(min_scs.groupby('Subregion')['WOY'].describe())


In [ ]:
# Look at which sites had AARs < 0.1
aar_lt_p1 = min_scs.loc[min_scs['AAR'] < 0.1]
print(len(aar_lt_p1))
aar_lt_p1

In [ ]:
# Print differences in AARs using full time series and only September observations
print("Median AAR difference for all sites:", (min_scs['AAR_Sept']-min_scs['AAR']).median())
print("Median differences by subregion:")
print(min_scs.groupby('Subregion')['AAR_Sept'].median() - min_scs.groupby('Subregion')['AAR'].median())

In [ ]:
scs_MCs_fn = os.path.join(scm_path, 'analysis', 'median_snow_cover_stats_MC.nc')
scs_MCs = xr.open_dataset(scs_MCs_fn)

# AARs
min_aar = scs_MCs.AAR.min(dim='WOY')
q0 = min_aar.quantile(q=0, dim='MC_run')
q100 = min_aar.quantile(q=1, dim='MC_run')
range = q100 - q0
std_dev = min_aar.std(dim='MC_run')  

# Average IQR and standard deviation across all sites
mean_range = range.mean(dim='RGIId')
mean_std_dev = std_dev.mean(dim='RGIId')
# Print results
print("AAR:")
print(f"Mean range across all sites: {mean_range.values}")
print(f"Mean standard deviation across sites: {mean_std_dev.values}")

# Snow minima timings
woy_min_aar = scs_MCs['WOY'].isel(WOY=scs_MCs.AAR.argmin(dim='WOY'))
q0_woy = woy_min_aar.quantile(q=0, dim='MC_run')
q25_woy = woy_min_aar.quantile(q=0.25, dim='MC_run')
q75_woy = woy_min_aar.quantile(q=0.75, dim='MC_run')
q100_woy = woy_min_aar.quantile(q=1, dim='MC_run')
range_woy = q100_woy - q0_woy
iqr_woy = q75_woy - q25_woy
std_dev_woy = woy_min_aar.std(dim='MC_run')
# Average IQR and standard deviation for WOY across all sites
mean_range_woy = range_woy.mean(dim='RGIId')
mean_iqr_woy = iqr_woy.mean(dim='RGIId')
mean_std_dev_woy = std_dev_woy.mean(dim='RGIId')
# Print results
print("\nSnow minima timings:")
print(f"Mean WOY range across sites: {mean_range_woy.values}")
print(f"Mean WOY IQR across sites: {mean_iqr_woy.values}")
print(f"Mean WOY standard deviation across sites: {mean_std_dev_woy.values}")


## Figure 3. Observed vs. modeled SMB

In [3]:
# Load merged monthly transient and annual ELAs 
slas_fn = os.path.join(scm_path, 'analysis', 'monthly_SLAs_modeled_observed_merged.csv')
slas = pd.read_csv(slas_fn)
slas['Date'] = pd.to_datetime(slas['Date'])
slas['Month'] = pd.DatetimeIndex(slas['Date']).month
elas_fn = os.path.join(scm_path, 'analysis', 'annual_ELAs_modeled_observed_merged.csv')
elas = pd.read_csv(elas_fn)
print('Merged ELAs and SLAs loaded')

# Load AOIs
aois_fn = os.path.join(scm_path, 'analysis', 'all_aois.shp')
aois = gpd.read_file(aois_fn)
aois[['O1Region', 'O2Region']] = aois[['O1Region', 'O2Region']].astype(int)
print('Compiled AOIs loaded')

# Load climate clusters
clusters_fn = os.path.join(scm_path, 'analysis', 'climate_clusters.csv')
clusters = pd.read_csv(clusters_fn)
print('Clusters loaded')

# Load observed and modeled melt factors of snow
fsnow_obs_fn = os.path.join(scm_path, 'analysis', 'fsnow_observed.csv')
fsnow_obs = pd.read_csv(fsnow_obs_fn)
fsnow_mod_fn = os.path.join(scm_path, 'analysis', 'fsnow_modeled.csv')
fsnow_mod = pd.read_csv(fsnow_mod_fn)
fsnow = fsnow_obs.merge(fsnow_mod, on='RGIId')
# adjust units for interpretability
fsnow['fsnow_obs_mm/C/d'] = fsnow['fsnow_obs_m/C/d'] * 1e3
fsnow['fsnow_mod_mm/C/d'] = fsnow['fsnow_mod_m/C/d'] * 1e3
print('Melt factors of snow loaded')

# Load SMB at observed SLAs
smb_at_slas_fn = os.path.join(scm_path, 'analysis', 'monthly_SLAs_modeled.csv')
smb_at_slas = pd.read_csv(smb_at_slas_fn)
smb_at_slas['Date'] = pd.to_datetime(smb_at_slas['Date'])
print('SMB at SLAs loaded')

# Add subregion, centroid coordinates, and climate cluster columns
fsnow = fsnow.merge(aois[['RGIId', 'Subregion', 'CenLon', 'CenLat']], on='RGIId')
fsnow = fsnow.merge(clusters[['RGIId', 'clustName']], on='RGIId')
fsnow['clustName'] = pd.Categorical(fsnow['clustName'], categories=cluster_order, ordered=True)
fsnow.sort_values(by='clustName', inplace=True)
slas = slas.merge(aois[['RGIId', 'Subregion', 'CenLon', 'CenLat']], on='RGIId')
slas = slas.merge(clusters[['RGIId', 'clustName']], on='RGIId')
elas = elas.merge(aois[['RGIId', 'Subregion', 'CenLon', 'CenLat']], on='RGIId')
elas = elas.merge(clusters[['RGIId', 'clustName']], on='RGIId')
smb_at_slas = smb_at_slas.merge(aois[['RGIId', 'Subregion', 'CenLon', 'CenLat']], on='RGIId')
smb_at_slas = smb_at_slas.merge(clusters[['RGIId', 'clustName']], on='RGIId')
print('Added columns for plotting')

Merged ELAs and SLAs loaded
Compiled AOIs loaded
Clusters loaded
Melt factors of snow loaded
SMB at SLAs loaded


NameError: name 'cluster_order' is not defined

In [ ]:
# Plot
fontsize=12
plt.rcParams.update({'font.size':fontsize, 'font.sans-serif': 'Arial'})
fig, ax = plt.subplots(2, 2, figsize=(10,8), gridspec_kw=dict(width_ratios=[1.5,1]))
ax = ax.flatten()

### a) Monthly snowline differences
slas['SLA_mod-obs_m'] = slas['SLA_mod_m'] - slas['SLA_obs_m']
sns.boxplot(data=slas, x='Month', y='SLA_mod-obs_m', hue='clustName', hue_order=cluster_order,
            palette=cluster_cmap_dict, saturation=1, showfliers=False,
            boxprops=dict(linewidth=1, edgecolor='k'), whiskerprops=dict(linewidth=1, color='k'), 
            ax=ax[0])
ax[0].set_xticks(np.arange(0,5))
ax[0].set_xticklabels(['May', 'Jun', 'Jul', 'Aug', 'Sep'])
ax[0].set_ylim(-700, 1200)
ax[0].set_title('a) Modeled $-$ observed snowline altitudes')
ax[0].set_ylabel('Snowline altitude difference [m]')
# add minor grid lines
ax[0].xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(0.5))
ax[0].grid(which='minor')
# remove minor ticks
ax[0].tick_params(axis='x', which='minor', bottom=False)

### b) ELA differences
elas['ELA_mod-obs_m'] = elas['ELA_mod_m'] - elas['ELA_obs_m']
sns.kdeplot(data=elas, y='ELA_mod-obs_m', hue='clustName', hue_order=cluster_order, fill=True, alpha=0.1,
            linewidth=2, palette=cluster_cmap_dict, ax=ax[1])
# Add median lines
# xmin, xmax = ax[1].get_xlim()
# for clustName in elas_merged_annual['clustName'].drop_duplicates().values:
#     med = elas_merged_annual.loc[elas_merged_annual['clustName']==clustName, 'ELA_mod-obs_m'].median()
#     ax[1].axhline(y=med, xmin=0, xmax=1, color=cluster_cmap_dict[clustName], linestyle='-', linewidth=1)
ax[1].set_ylim(ax[0].get_ylim())
ax[1].set_ylabel('')
ax[1].set_title('b) Modeled $-$ observed ELAs')
ax[1].set_ylabel('ELA difference [m]')
ax[1].set_xticks([])

### c) Modeled SMB at remotely-sensed snowlines
smb_at_slas['Month'] = smb_at_slas['Date'].dt.month
sns.boxplot(data=smb_at_slas, x='Month', y='SMB_at_obs_SLA_mwe', hue='clustName', hue_order=cluster_order,
            palette=cluster_cmap_dict, saturation=1, showfliers=False, legend=False,
            boxprops=dict(linewidth=1, edgecolor='k'), whiskerprops=dict(linewidth=1, color='k'), 
            ax=ax[2])
ax[2].set_xlabel('Month')
ax[2].set_xticks(np.arange(4,9))
ax[2].set_xticklabels(['May', 'Jun', 'Jul', 'Aug', 'Sep'])
ax[2].set_xlim(3.5, 8.5)
ax[2].set_ylabel('SMB [m.w.e.]')
ax[2].set_title('c) Modeled SMB at observed snowline altitude')
# add minor grid lines
ax[2].xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(0.5))
ax[2].grid(which='minor')
# remove minor ticks
ax[2].tick_params(axis='x', which='minor', bottom=False)

### d) Violin plots of observed and modeled melt factors
lw = 0.5
fsnow_melt = pd.melt(fsnow, id_vars=['RGIId', 'Subregion', 'CenLat', 'CenLon', 'clustName'], 
                     value_vars=['fsnow_obs_mm/C/d', 'fsnow_mod_mm/C/d'], 
                     var_name='fsnow_type', value_name='fsnow_mm/C/d')
sns.violinplot(fsnow_melt, x='clustName', y='fsnow_mm/C/d', hue='fsnow_type', 
               split=True, inner='quart', legend=False, ax=ax[3])
# Customize the colors
for violin, color in zip(plt.gca().collections[::2], cluster_cmap_dict.values()):
    violin.set_facecolor(color)
    violin.set_edgecolor('k')
    violin.set_linewidth(lw)
for violin in plt.gca().collections[1::2]:
    violin.set_facecolor('None')
    violin.set_edgecolor('k')
    violin.set_linewidth(lw)
# Plot dummy points for legend
xlim, ylim = plt.gca().get_xlim(), plt.gca().get_ylim()
ax[3].plot(-5,-5, 's', markersize=15, markerfacecolor='gray', markeredgecolor='k', markeredgewidth=lw, label='Observed')
ax[3].plot(-5,-5, 's', markersize=15, markerfacecolor='w', markeredgecolor='k', markeredgewidth=lw, label='Modeled')
ax[3].legend(loc='lower left', frameon=False, labelspacing=1.0, handletextpad=0.3)
ax[3].set_xlim(xlim)
ax[3].set_ylim(ylim)
# ax[3].set_xticks([])
ax[3].set_xticklabels([])
ax[3].set_xlabel('Climate cluster')
ax[3].set_ylabel('$f_{snow}$ [mm $^{\circ}$C$^{-1}$ d$^{-1}$]')
ax[3].set_title('d) Degree day factors of snow')

# Add legend
handles, labels = ax[0].get_legend_handles_labels()
ax[0].legend().remove()
ax[1].legend().remove()
fig.legend(handles, labels, loc='lower center', ncols=5, frameon=False,
           bbox_to_anchor=[0.43, -0.05, 0.2, 0.2], labelspacing=0.6, handletextpad=0.3)

# Add text label and line at 0
text_labels = ['a', 'b']
for i, axis in enumerate(ax):
    axis.axhline(0, color='k')
    # axis.text((axis.get_xlim()[1]-axis.get_xlim()[0])*0.9 + axis.get_xlim()[0],
    #           (axis.get_ylim()[1]-axis.get_ylim()[0])*0.9 + axis.get_ylim()[0],
    #           text_labels[i], fontsize=fontsize+4, fontweight='bold')
    
# fig.subplots_adjust(wspace=0.2)
fig.tight_layout()
plt.show()

# Save to file
fig_fn = os.path.join(figures_out_path, 'fig03_modeled_observed_SMB_differences.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)


In [ ]:
# SLA stats
print(slas.groupby(['Month'])['SLA_mod-obs_m'].describe())
slas.groupby(['clustName', 'Month'])['SLA_mod-obs_m'].describe()

In [ ]:
# ELA stats
print(elas['ELA_mod-obs_m'].describe())
elas.groupby('clustName')['ELA_mod-obs_m'].describe().sort_values(by='50%')

In [ ]:
# fsnow stats
print('OBSERVED')
print(fsnow['fsnow_obs_mm/C/d'].describe())
print(fsnow.groupby('clustName')['fsnow_obs_mm/C/d'].describe())

print('\nMODELED')
print(fsnow['fsnow_mod_mm/C/d'].describe())
print(fsnow.groupby('clustName')['fsnow_mod_mm/C/d'].describe())

print('\nMODELED - OBSERVED:')
fsnow['fsnow_mod-obs_mm/C/d'] = fsnow['fsnow_mod_mm/C/d'] - fsnow['fsnow_obs_mm/C/d']
sns.boxplot(fsnow, x='clustName', y='fsnow_mod-obs_mm/C/d', hue='clustName', 
            palette=cluster_cmap_dict, showfliers=False)
plt.show()
print(fsnow['fsnow_mod-obs_mm/C/d'].describe())
fsnow.groupby('clustName')['fsnow_mod-obs_mm/C/d'].describe()

## Figure S1. Sites distributions

In [ ]:
# Load analyzed glacier boundaires
aois_fn = os.path.join(scm_path, 'analysis', 'all_aois.shp')
aois = gpd.read_file(aois_fn)
cols = ['O1Region', 'O2Region', 'Zmed', 'Aspect', 'Slope', 'Area']
for col in cols:
    aois[col] = aois[col].astype(float)
aois['Subregion'] = ''
for o1region, o2region in aois[['O1Region', 'O2Region']].drop_duplicates().values:
    subregion_name, color = f.determine_subregion_name_color(o1region, o2region)
    aois.loc[(aois['O1Region']==o1region) & (aois['O2Region']==o2region), 'Subregion'] = subregion_name

    
# Load all glacier boundaries in O1 regions 1 and 2
rgi_path = '/Volumes/LaCie/raineyaberle/Research/PhD/GIS_data/RGI/'
rgi_fns = ['01_rgi60_Alaska/01_rgi60_Alaska.shp',
           '02_rgi60_WesternCanadaUS/02_rgi60_WesternCanadaUS.shp']
rgi = gpd.GeoDataFrame()
for rgi_fn in rgi_fns:
    file = gpd.read_file(os.path.join(rgi_path, rgi_fn))
    rgi = pd.concat([rgi, file])
rgi[['O1Region', 'O2Region']] = rgi[['O1Region', 'O2Region']].astype(int)
# Add column for subregion name
rgi['Subregion'] = ''
for o1region, o2region in rgi[['O1Region', 'O2Region']].drop_duplicates().values:
    subregion_name, color = f.determine_subregion_name_color(o1region, o2region)
    rgi.loc[(rgi['O1Region']==o1region) & (rgi['O2Region']==o2region), 'Subregion'] = subregion_name


In [ ]:
# Define plotting variables
columns = ['Zmed', 'Aspect', 'Slope', 'Area']
xlabels = ['Median elevation [m]', 'Aspect [degrees]', 'Slope [degrees]', 'Area [km$^2$]']
bins_list = [np.linspace(0, 361, num=20), # Aspect
             np.linspace(0, 51, num=20), # Slope
             np.linspace(0, 300, num=50)] # Area

# Set up figure
plt.rcParams.update({'font.sans-serif': 'Arial', 'font.size': 12})
fig, ax = plt.subplots(len(subregion_order)+1, 4, figsize=(12, (len(subregion_order)+1)*1.5))
aois_color = '#b35806'

# All subregions
for j, (column, xlabel) in enumerate(zip(columns, xlabels)):
    if column=='Zmed': 
        bins = np.linspace(rgi['Zmed'].min(), rgi['Zmed'].max(), num=20)
    else:
        bins  = bins_list[j-1]
    ax[0,j].hist(rgi[column].values, bins=bins, facecolor='k', alpha=0.6)
    ax[0,j].set_title(xlabel)
    ax2 = ax[0,j].twinx()
    ax2.hist(aois[column].values, bins=bins, facecolor=aois_color, alpha=0.6)
    ax2.set_yticks(ax2.get_yticks())
    ax2.set_yticklabels(ax2.get_yticklabels(), color=aois_color)
    ax2.spines['right'].set_color(aois_color)
    ax2.tick_params(axis='y', color=aois_color)
ax[0,0].set_ylabel('All regions', fontweight='bold')

# Individual subregions
for i, subregion in enumerate(subregion_order):
    # Subset glaciers
    aois_subregion = aois.loc[aois['Subregion']==subregion]
    rgi_subregion = rgi.loc[rgi['Subregion']==subregion]

    # Plot all glaciers in subregion
    for j, (column, xlabel) in enumerate(zip(columns, xlabels)):
        if column=='Zmed':
            bins = np.linspace(rgi_subregion['Zmed'].min(), rgi_subregion['Zmed'].max(), num=20)
        else:
            bins = bins_list[j-1]
        ax[i+1,j].hist(rgi_subregion[column].values, bins=bins, facecolor='k', alpha=0.6)
        if j==0:
            ax[i+1,j].set_ylabel(subregion)
        ax2 = ax[i+1,j].twinx()
        ax2.hist(aois_subregion[column].values, bins=bins, facecolor=aois_color, alpha=0.6)
        ax2.set_yticks(ax2.get_yticks())
        ax2.set_yticklabels(ax2.get_yticklabels(), color=aois_color)
        ax2.spines['right'].set_color(aois_color)
        ax2.tick_params(axis='y', color=aois_color)

fig.tight_layout()
plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'figS1_site_distributions.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

## Figure S2. Snowline altitude uncertainty analysis

In [ ]:
# Example site ID
rgi_id = 'RGI60-01.01104'
epsg_utm = "EPSG:32606"

# Load SLA bounds
sla_bounds_fn = os.path.join(scm_path, 'analysis', 'SLA_uncertainty_analysis.csv')
sla_bounds_df = pd.read_csv(sla_bounds_fn)

# Load all AOIs
aois_fn = os.path.join(scm_path, 'analysis', 'all_aois.shp')
aois = gpd.read_file(aois_fn)
# Add elevation range column
aois['Zrange'] = aois['Zmax'] - aois['Zmin']
# Merge with SLA bounds
sla_bounds_df = sla_bounds_df.merge(aois[['RGIId', 'Zrange']], on='RGIId')

# Load snow cover stats
scs_fn = os.path.join(scm_path, 'study-sites', rgi_id, f"{rgi_id}_snow_cover_stats.csv")
scs = pd.read_csv(scs_fn)
scs = scs.loc[scs['AAR'] < 1].reset_index(drop=True) # remove rows where AAR = 1
# Load DEM
dem_fn = glob.glob(os.path.join(scm_path, 'study-sites', rgi_id, 'DEMs', '*.tif'))[0]
dem = rxr.open_rasterio(dem_fn).isel(band=0)
dem = xr.where((dem < -1e3) | (dem > 1e4), np.nan, dem)
dem = dem.rio.write_crs("EPSG:4326")
# Load AOI
aoi_fn = os.path.join(scm_path, 'study-sites', rgi_id, 'AOIs', f"{rgi_id}_outline.shp")
aoi = gpd.read_file(aoi_fn)
# Clip DEM to AOI
dem = dem.rio.clip(aoi.geometry)
# Choose an image with some masking and relatively low transient AAR for demonstration
sc = scs.loc[(scs['datetime']=='2019-07-03T20:18:52') & (scs['source']=='Sentinel-2_SR')].reset_index(drop=True).iloc[0]
# sc = scs.loc[(scs['AAR'] < 0.5) & (scs['source']=='Sentinel-2_SR')].reset_index(drop=True).iloc[0]
classified_fn = glob.glob(os.path.join(scm_path, 'study-sites', rgi_id, 'classified', 
                                       f"{sc['datetime'].replace('-', '').replace(':','')}_{rgi_id}_{sc['source']}_classified.nc"))[0]
classified = rxr.open_rasterio(classified_fn).squeeze()
classified = xr.where(classified==-9999, np.nan, classified)
classified = classified.rio.write_crs("EPSG:4326")
classified_utm = classified.rio.reproject(epsg_utm)
classified_utm = xr.where(classified_utm > 1e30, np.nan, classified_utm)

# Create binary snow image
snow_binary = xr.where((classified==1) | (classified==2), 1, 0)
snow_binary = xr.where(np.isnan(classified), np.nan, snow_binary) # re-insert no data values

# Regrid DEM to classified image grid
dem = dem.rio.reproject_match(classified)
dem = xr.where(dem > 1e4, np.nan, dem)
dem = dem.rio.write_crs("EPSG:4326")
dem_utm = dem.rio.reproject(epsg_utm)

# Calculate lower and upper bounds of snowline altitude
if sc['source']=='Landsat':
    dx=30
elif 'Sentinel-2' in sc['source']:
    dx=10
sla, sla_upper_bound, sla_lower_bound = f.calculate_sla_bounds(sc, dem, snow_binary, dx=dx, verbose=True)   
dem_utm = xr.where(dem_utm > 1e30, np.nan, dem_utm)
# Define colormap for classified images
cmap_dict = {"Snow": "#4eb3d3",  "Shadowed_snow": "#4eb3d3", "Ice": "#084081", "Rock": "#fe9929", "Water": "#969696"}
colors = []
for key in list(cmap_dict.keys()):
    color = list(matplotlib.colors.to_rgb(cmap_dict[key]))
    if key=='Rock':
        color += [0.5]
    colors.append(color)
cmap = matplotlib.colors.ListedColormap(colors)
cmap

In [ ]:
# Set up figure
fontsize=11
lw=1.5
plt.rcParams.update({'font.size':fontsize, 'font.sans-serif': 'Arial'})
gs = matplotlib.gridspec.GridSpec(3,2, height_ratios=[1, 1.5, 1.5])
fig = plt.figure(figsize=(8,12))
ax = [fig.add_subplot(gs[0,0]), fig.add_subplot(gs[0,1]),
      fig.add_subplot(gs[1,:]),
      fig.add_subplot(gs[2,:])]

# DEM
im = ax[0].imshow(dem_utm.data, cmap='terrain', 
                  extent=(np.min(dem_utm.x)/1e3, np.max(dem_utm.x)/1e3, np.min(dem_utm.y)/1e3, np.max(dem_utm.y)/1e3))
cbar = fig.colorbar(im, ax=ax[0], shrink=0.9, label='Elevation [m]')
ax[0].set_xlabel('Easting [km]')
ax[0].set_ylabel('Northing [km]')

# classified image
im = ax[1].imshow(classified_utm.data, cmap=cmap, clim=(0.5,5.5),
                  extent=(np.min(classified_utm.x)/1e3, np.max(classified_utm.x)/1e3, np.min(classified_utm.y)/1e3, np.max(classified_utm.y)/1e3))
cbar = fig.colorbar(im, ax=ax[1], shrink=0.9, ticks=[2, 3, 4, 5])
cbar.ax.set_yticklabels(['Snow', 'Ice/firn', 'Rock', 'Water'])
cbar.ax.set_ylim(1.5,5.5)
ax[1].set_yticklabels([])
ax[1].set_xlabel('Easting [km]')
ax[1].set_ylabel('')

# SLA contours
x_mesh, y_mesh = np.meshgrid(np.divide(dem_utm.x.data, 1e3), np.divide(dem_utm.y.data, 1e3))
for axis in ax[0:2]:
    axis.contour(x_mesh, y_mesh, dem_utm.data, levels=[sla], colors='k', linewidth=lw)

# histograms and bounds
bin_edges = np.arange(700, 1481, step=10)
bin_centers = (bin_edges[1:] + bin_edges[0:-1]) / 2
# all elevations
counts, _ = np.histogram(dem_utm.data, bins=bin_edges)
areas = counts * dx**2 / 1e6 # km^2
ax[2].bar(bin_centers, areas, width=bin_edges[1]-bin_edges[0], facecolor='gray', 
          edgecolor='k', alpha=0.5, linewidth=lw-1, label='All elevations')
# snow-covered elevations
dem_snow = xr.where(classified_utm==1, dem_utm, np.nan)
counts, _ = np.histogram(dem_snow.data, bins=bin_edges)
areas = counts * dx**2 / 1e6 # km^2
ax[2].bar(bin_centers, areas, width=bin_edges[1]-bin_edges[0], facecolor=cmap_dict['Snow'], 
          edgecolor='k', alpha=1.0, linewidth=lw-1, label='Snow-covered elevations')
ax[2].axvline(sla, color='k', linewidth=lw, label='Original SLA')
ax[2].axvline(sla_lower_bound, color='k', linestyle='--', linewidth=lw, label='SLA lower bound')
ax[2].axvline(sla_upper_bound, color='k', linestyle=':', linewidth=lw, label='SLA upper bound')
ax[2].text(850, 0.28, "Snow-covered area \nbelow SLA = 0.78 km$^2$", fontsize=9, ha='center',
           bbox=dict(facecolor='w', edgecolor='None', alpha=0.7))
ax[2].text(1350, 0.28, "Snow-free area \nabove SLA = 0.66 km$^2$", fontsize=9, ha='center')
ax[2].set_xlim(np.min(bin_edges)-20, np.max(bin_edges)+20)
ax[2].set_xlabel('Elevation [m]')
ax[2].set_ylabel('Area [km$^2$]')
ax[2].legend(loc='upper left')

# histogram of SLA ranges
ax[3].hist(sla_bounds_df['SLA_bounds_range_m'], bins=np.linspace(0, 1000, num=101), 
           facecolor='gray', alpha=0.9, edgecolor='k', linewidth=0.5)
range_median = np.nanmedian(sla_bounds_df['SLA_bounds_range_m'])
range_mean = np.nanmean(sla_bounds_df['SLA_bounds_range_m'])
range_p25 = np.nanpercentile(sla_bounds_df['SLA_bounds_range_m'], 25)
range_p75 = np.nanpercentile(sla_bounds_df['SLA_bounds_range_m'], 75)
ax[3].axvline(range_median, color='k', linewidth=lw, 
              label=f"Median = {int(range_median)} m")
ax[3].axvline(range_mean, color='k', linewidth=lw, linestyle='--', 
              label=f"Mean = {int(range_mean)} m")
ax[3].fill_between([range_p25, range_p75], [0,0], [400,400], color='k', alpha=0.1, edgecolor='k', linewidth=lw+1,
                   label=f"IQR = {int(range_p25)}–{int(range_p75)} m")
ax[3].legend(loc='center right')
ax[3].set_xlabel('Range of all SLA bounds [m]')
ax[3].set_ylabel('Counts')
ax[3].set_xlim(0, 600)
ax[3].set_ylim(0,360)
# add panel labels
labels = ['a', 'b', 'c', 'd']
for i, axis in enumerate(ax):
    if i < 2:
        xscale=0.85
        yscale=0.9
    else:
        xscale=0.95
        yscale=0.9
    axis.text(xscale, yscale, labels[i], transform=axis.transAxes, fontsize=fontsize+4, fontweight='bold')

fig.tight_layout()
plt.show()

# Save figure to file
fig_fn = os.path.join(figures_out_path, 'figS2_SLA_uncertainties.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)


## Figure S3. Median snow cover minima sampling

In [ ]:
# Set up figure
gs = matplotlib.gridspec.GridSpec(2,2, height_ratios=[1.5, 1])
fig = plt.figure(figsize=(7,8))
ax = [fig.add_subplot(gs[0,:]), 
      fig.add_subplot(gs[1,0]), fig.add_subplot(gs[1,1])]

# Plot an example Monte Carlo simulation at South Cascade Glacier
rgi_id = 'RGI60-02.18778'
scs_fn = os.path.join(scm_path, 'study-sites', rgi_id, f"{rgi_id}_snow_cover_stats.csv")
scs = pd.read_csv(scs_fn)
scs['datetime'] = pd.to_datetime(scs['datetime'], errors='coerce')
scs.dropna(subset=['datetime'], inplace=True)
scs['WOY'] = scs['datetime'].dt.isocalendar().week
scs['Year'] = scs['datetime'].dt.isocalendar().year
scs['Year'] = pd.Categorical(scs['Year'])
scs['SCA_km2'] = scs['SCA_m2'] / 1e6
# Plot
sns.scatterplot(scs, x='WOY', y='AAR', hue='Year', palette='viridis', s=10, legend=True, ax=ax[0])
# Number of samples per simulation
sample_fraction = 0.8
nsamp = int(len(scs) * sample_fraction)
nMC = 100
# Monte Carlo simulations
results = pd.DataFrame()
for i in range(nMC):
    sampled_indices = np.random.choice(scs.index, size=nsamp, replace=False)
    scs_MC = scs.loc[sampled_indices].sort_values(by='datetime')
    # Calculate weekly medians for AAR, SCA, and ELA
    weekly_medians = scs_MC.groupby('WOY')[['AAR', 'SCA_km2', 'ELA_from_AAR_m']].median()
    weekly_medians['MC_run'] = i    
    # Plot
    ax[0].plot(weekly_medians.index, weekly_medians['AAR'], '-', color='gray', linewidth=0.1)
    results = pd.concat([results, weekly_medians])
# Dummy line for legend
ax[0].plot([0,0], [0,0], '-', color='gray', linewidth=2, label='MC simulation')
# Estimate median AAR and snow minimum timing
medians = results.groupby('WOY')['AAR'].median()
value = medians.loc[medians==min(medians)]
ax[0].scatter(value.index[0], medians.min(), marker='x', s=50, color='k', linewidth=2, label='Median AAR', zorder=5)
handles, labels = ax[0].get_legend_handles_labels()
ax[0].set_ylabel('Transient accumulation area ratio')
ax[0].set_xlabel('Week of year')
ax[0].set_xlim(16,45)
ax[0].set_ylim(-0.1,1.1)
# add legend
ax[0].legend().remove()
handles, labels = ax[0].get_legend_handles_labels()
ax[0].legend(handles[:11:2] + handles[-2:], labels[:11:2] + labels[-2:], loc='lower left', markerscale=2)

# Plot AAR and snow minima timing distributions for all sites
median_scs_fn = os.path.join(scm_path, 'analysis', 'median_snow_cover_stats_MC.nc')
median_scs = xr.open_dataset(median_scs_fn)
aar_woy_mads = pd.DataFrame()
# iterate over site names in median snow cover stats dataframe
for rgi_id in tqdm(median_scs.RGIId.values):
    # Calculate AAR median and MAD across MC simulations
    aar_median = float(median_scs.sel(RGIId=rgi_id).AAR.min(dim='WOY').median().values)
    aar_mad = median_abs_deviation(median_scs.sel(RGIId=rgi_id).AAR.min(dim='WOY').values)
    Imins = median_scs.sel(RGIId=rgi_id).AAR.argmin(dim='WOY').values
    woy_mad = median_abs_deviation(median_scs.WOY.values[Imins])
    # Add to dataFrame
    df = pd.DataFrame({'RGIId': [rgi_id],
                       'AAR_MAD': [aar_mad],
                       'WOY_MAD': [woy_mad]})
    aar_woy_mads = pd.concat([aar_woy_mads, df], axis=0)
# Plot
ax[1].hist(aar_woy_mads['AAR_MAD'], bins=50, color='gray', edgecolor='k', linewidth=0.5)
ax[1].text(0.95, 0.7, f"Median = {np.round(np.nanmedian(df['AAR_MAD']),3)}", ha='right', transform=ax[1].transAxes)
ax[1].text(0.95, 0.63, f"Mean = {np.round(np.nanmean(df['AAR_MAD']),3)}", ha='right', transform=ax[1].transAxes)
ax[1].set_xlabel('AAR MAD')
ax[1].set_ylabel('Counts')
ax[2].hist(aar_woy_mads['WOY_MAD'], bins=50, color='gray', edgecolor='k', linewidth=0.5)
ax[2].text(0.95, 0.7, f"Median = {np.round(np.nanmedian(df['WOY_MAD']),3)}", ha='right', transform=ax[2].transAxes)
ax[2].text(0.95, 0.63, f"Mean = {np.round(np.nanmean(df['WOY_MAD']),3)}", ha='right', transform=ax[2].transAxes)
ax[2].set_xlabel('Snow minimum timing MAD [weeks]')

# add panel labels
labels = ['a', 'b', 'c']
for i, axis in enumerate(ax):
    axis.text(0.9, 0.92, labels[i], transform=axis.transAxes, fontsize=fontsize+2, fontweight='bold')


fig.tight_layout()
plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'figS3_AAR_MC_simulations.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

## Table S1. Study sites with subregion and climate cluster

In [ ]:
aois = gpd.read_file(aois_fn)
aois.rename(columns={'Subregion': 'Subregion name'}, inplace=True)
aois[['O1Region', 'O2Region']] = aois[['O1Region', 'O2Region']].astype(int)

# Add climate cluster
clusters = pd.read_csv(clusters_fn)
aois['Climate cluster'] = ''
for rgi_id in aois['RGIId'].drop_duplicates().values:
    aois.loc[aois['RGIId']==rgi_id, 'Climate cluster'] = clusters.loc[clusters['RGIId']==rgi_id, 'clustName']
aois.sort_values(by=['O1Region', 'O2Region'], inplace=True)

# Format as LaTeX table
columns = ['RGIId', 'O1Region', 'O2Region', 'Subregion name', 'Climate cluster']
aois = aois[columns]

# Save as Excel sheet
aois_xl_fn = aois_fn.replace('all_aois.shp', 'TableS1_study_sites.xlsx')
aois.to_excel(aois_xl_fn, index=False)
print('Table saved as Excel spreadsheet:', aois_xl_fn)
aois


## README figures

### Median weekly trends

In [ ]:
# Set up figure
plt.rcParams.update({'font.size':14, 'font.sans-serif': "Arial"})
fig, ax = plt.subplots(1, 3, figsize=(15,5))

# Load snow cover stats
rgi_id = 'RGI60-02.18778'
scs_fn = os.path.join(scm_path, 'study-sites', rgi_id, f"{rgi_id}_snow_cover_stats.csv")
scs = pd.read_csv(scs_fn)
scs['datetime'] = pd.to_datetime(scs['datetime'], errors='coerce')
scs.dropna(subset=['datetime'], inplace=True)
scs['WOY'] = scs['datetime'].dt.isocalendar().week
scs['Year'] = scs['datetime'].dt.isocalendar().year
scs['Year'] = pd.Categorical(scs['Year'])
scs['SCA_km2'] = scs['SCA_m2'] / 1e6
# Plot
sns.scatterplot(scs, x='WOY', y='AAR', hue='Year', palette='viridis', size=0.5, legend=False, ax=ax[0])
sns.scatterplot(scs, x='WOY', y='SCA_km2', hue='Year', palette='viridis', size=0.5, legend=False, ax=ax[1])
sns.scatterplot(scs, x='WOY', y='ELA_from_AAR_m', hue='Year', palette='viridis', size=0.5, legend=False, ax=ax[2])


# Number of samples per simulation
sample_fraction = 0.8
nsamp = int(len(scs) * sample_fraction)
nMC = 100

# Monte Carlo simulations
results = pd.DataFrame()
for i in range(nMC):
    sampled_indices = np.random.choice(scs.index, size=nsamp, replace=False)
    scs_MC = scs.loc[sampled_indices].sort_values(by='datetime')

    # Calculate weekly medians for AAR, SCA, and ELA
    weekly_medians = scs_MC.groupby('WOY')[['AAR', 'SCA_km2', 'ELA_from_AAR_m']].median()
    weekly_medians['MC_run'] = i    
    # Plot
    if i==0:
        label = 'MC simulations'
    else:
        label = '_nolegend'
    ax[0].plot(weekly_medians.index, weekly_medians['AAR'], '-', color='gray', linewidth=0.1, label=label)
    ax[1].plot(weekly_medians.index, weekly_medians['SCA_km2'], '-', color='gray', linewidth=0.1)
    ax[2].plot(weekly_medians.index, weekly_medians['ELA_from_AAR_m'], '-', color='gray', linewidth=0.1)
    results = pd.concat([results, weekly_medians])
    
# Estimate median AAR and snow minimum timing
for i, column in enumerate(['AAR', 'SCA_km2', 'ELA_from_AAR_m']):
    medians = results.groupby('WOY')[column].median()
    if column!='ELA_from_AAR_m':
        value = medians.loc[medians==min(medians)]
    else:
        value = medians.loc[medians==max(medians)]
    ax[i].axvline(value.index[0], color='k', label='Minimum snow cover median')
handles, labels = ax[0].get_legend_handles_labels()
ax[0].set_ylabel('Transient accumulation area ratio')
ax[1].set_ylabel('Snow covered area [km$^2$]')
ax[2].set_ylabel('Snowline altitude [m]')
for axis in ax:
    axis.set_xlabel('Week of year')
fig.tight_layout()
plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'weekly_median_trends_example.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

## AGU24 figures

### Abstract figure

In [ ]:
cmap = sns.color_palette('mako', n_colors=len(subregion_order)+2)

# -----Load median AARs for all sites
min_snow_cover_stats_fn = os.path.join(scm_path, 'results', 'min_snow_cover_stats.csv')
min_snow_cover_stats = pd.read_csv(min_snow_cover_stats_fn)
# Sort subregions
min_snow_cover_stats['order'] = ''
for i, subregion in enumerate(subregion_order):
    min_snow_cover_stats.loc[min_snow_cover_stats['Subregion']==subregion, 'order'] = i
    min_snow_cover_stats.loc[min_snow_cover_stats['Subregion']==subregion, 'color'] = matplotlib.colors.to_hex(cmap[i])
min_snow_cover_stats = min_snow_cover_stats.sort_values(by='order')
print('Median AARs loaded from file')

# # -----Load RGI O2 Regions
# rgi_O2_fn = os.path.join(scm_path, '..', 'GIS_data', 'RGI', 'RGIv7_02Regions', 
#                                 'RGI2000-v7.0-o2regions-Alaska-westernCanadaUS_clipped_to_country_outlines.shp')
# rgi_O2 = gpd.read_file(rgi_O2_fn)
# # remove Brooks Range
# rgi_O2 = rgi_O2.loc[rgi_O2['o2region']!='01-01']
# # add subregion name and color column
# rgi_O2[['Subregion', 'color']] = '', ''
# for i, o1o2 in enumerate(rgi_O2['o2region'].drop_duplicates().values):
#     o1 = int(o1o2[0:2])
#     o2 = int(o1o2[3:])
#     subregion_name, color = f.determine_subregion_name_color(o1, o2)
#     rgi_O2.loc[rgi_O2['o2region']==o1o2, 'Subregion'] = subregion_name
#     rgi_O2.loc[rgi_O2['o2region']==o1o2, 'color'] = dict(min_snow_cover_stats[['Subregion', 'color']].drop_duplicates().values)[subregion]
# print('RGI O2 regions loaded from file')

# # -----Load GTOPO30
# gtopo_fn = '/Users/raineyaberle/Research/PhD/GIS_data/GTOPO30_clip.tif'
# gtopo = rxr.open_rasterio(gtopo_fn)
# gtopo = xr.where(gtopo==-32768, np.nan, gtopo)
# print('GTOPO30 loaded from file')

# # -----Load classified image
# site_name = 'RGI60-01.00037'
# im_classified_fn = f'/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/study-sites/{site_name}/imagery/classified/20230802T152742_RGI60-01.00037_Sentinel-2_SR_classified.nc'
# im_classified = xr.open_dataset(im_classified_fn)
# print('Classified image loaded')

# # -----Load classified images colormap
# import json
# datasets_dict_fn = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/inputs-outputs/datasets_characteristics.json'
# datasets_dict = json.load(open(datasets_dict_fn))
# cmap_classified = matplotlib.colors.ListedColormap(datasets_dict['classified_image']['class_colors'].values())

# # -----Load Sentinel-2 image from GEE
# import math
# import wxee as wx
# import geedim as gd
# import ee
# ee.Initialize()
# def convert_wgs_to_utm(lon: float, lat: float):
#     utm_band = str((math.floor((lon + 180) / 6) % 60) + 1)
#     if len(utm_band) == 1:
#         utm_band = '0' + utm_band
#     if lat >= 0:
#         epsg_code = '326' + utm_band
#         return epsg_code
#     epsg_code = '327' + utm_band
#     return epsg_code
# # Load AOI
# aoi_fn = glob.glob(f'/Volumes/LaCie/raineyaberle/Research/PhD/snow_cover_mapping/study-sites/{site_name}/AOIs/*.shp')[0]
# aoi = gpd.read_file(aoi_fn)
# aoi_bounds = aoi.geometry[0].bounds
# region = ee.Geometry.Polygon([[aoi_bounds[0], aoi_bounds[1]], 
#                               [aoi_bounds[2], aoi_bounds[1]],
#                               [aoi_bounds[2], aoi_bounds[3]],
#                               [aoi_bounds[0], aoi_bounds[3]],
#                               [aoi_bounds[0], aoi_bounds[1]]])
# # Load image collection
# im_col = gd.MaskedCollection.from_name('COPERNICUS/S2_SR_HARMONIZED').search(start_date='2023-08-01',
#                                                                              end_date='2023-08-03',
#                                                                              region=region,
#                                                                              mask=True)
# im_ee = im_col.ee_collection.first()
# im_ee = im_ee.clip(region)
# im_ee = im_ee.select(['B4', 'B3', 'B2'])
# # Convert to xarray.Dataset
# im_xr = im_ee.wx.to_xarray(scale=30, crs='EPSG:4326')
# im_xr = xr.where(im_xr==im_xr.attrs['_FillValue'], np.nan, im_xr / 1e4)
# im_xr = im_xr.rio.write_crs('EPSG:4326')
# print('Sentinel-2 SR image loaded')

# # Reproject AOI and images to optimal UTM zone
# epsg_utm = convert_wgs_to_utm(aoi.geometry[0].centroid.coords.xy[0][0], aoi.geometry[0].centroid.coords.xy[1][0])
# aoi_utm = aoi.to_crs(f'EPSG:{epsg_utm}')
# im_xr = im_xr.rio.reproject(f'EPSG:{epsg_utm}')
# im_classified = im_classified.rio.write_crs("EPSG:4326")
# im_classified = im_classified.rio.reproject(f'EPSG:{epsg_utm}')
# im_classified = xr.where(im_classified < 1, np.nan, im_classified)

In [ ]:
# Set up figure
fontsize=14
plt.rcParams.update({
    "font.size": fontsize,
    "font.sans-serif": "Arial",
    # "font.family": "sans-serif",
    # "font.sans-serif": "Computer Modern Sans Serif",
    "text.usetex": False
})

fig = plt.figure(figsize=(12,12))
gs = matplotlib.gridspec.GridSpec(2, 2, figure=fig, height_ratios=[1,2])
ax = [fig.add_subplot(gs[2:]),
      fig.add_subplot(gs[0]),
      fig.add_subplot(gs[1])]

# ----- Study sites
# GTOPO hillshade
ls = matplotlib.colors.LightSource(azdeg=90, altdeg=45)
ax[0].imshow(ls.hillshade(gtopo.data[0], vert_exag=0.002), cmap='gray', alpha=0.5,
             extent=(np.min(gtopo.x.data), np.max(gtopo.x.data), 
                     np.min(gtopo.y.data), np.max(gtopo.y.data)))
# RGI O2 region outlines
color = '#525252'
rgi_O2.plot(ax=ax[0], alpha=1.0, facecolor='None', edgecolor=color, linewidth=1)
ax[0].set_yticks(np.linspace(45, 65, num=6))
ax[0].set_xlim(-167, -112)
ax[0].set_ylim(46, 67)
ax[0].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')
ax[0].set_aspect(2.2)
# Median AARs
sns.scatterplot(data=min_snow_cover_stats, x='CenLon', y='CenLat', edgecolor='w', linewidth=0.5, 
                hue='Subregion', hue_order=subregion_order, palette=dict(min_snow_cover_stats[['Subregion', 'color']].drop_duplicates().values), 
                alpha=1, size='AAR_P50_min', sizes=(2,100), ax=ax[0])
handles, labels = ax[0].get_legend_handles_labels()
Ikeep = np.argwhere(['0.' in x for x in np.array(labels)]).flatten()
handles, labels = [handles[i] for i in Ikeep], [labels[i] for i in Ikeep] 
ax[0].legend(handles, labels, loc='lower left', title='2013–2023 median AAR', bbox_to_anchor=[0.2, 0.05, 0.2, 0.2])
# Add region labels and arrows
fontweight = 'bold'
ax[0].text(-163, 56, 'Aleutians', color=color, rotation=35, fontsize=fontsize-1, fontweight=fontweight)
ax[0].text(-158, 62.3, 'Alaska Range', color=color, rotation=0, fontsize=fontsize-1, fontweight=fontweight)
ax[0].text(-147.9, 57.8, 'W. Chugach \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-1, fontweight=fontweight)
ax[0].arrow(-147.6, 58.8, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-141.7, 57.7, 'St. Elias \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-1, fontweight=fontweight)
ax[0].arrow(-141.5, 58.7, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-139.6, 56.4, 'N. Coast \nRanges', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-1, fontweight=fontweight)
ax[0].arrow(-137.3, 56.8, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-133, 51.3, 'N. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-1, fontweight=fontweight)
ax[0].arrow(-129.4, 51.4, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-129.7, 47, 'S. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-1, fontweight=fontweight)
ax[0].arrow(-126, 47.1, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax[0].text(-132, 64, 'N. Rockies', color=color, rotation=0, fontsize=fontsize-1, fontweight=fontweight)
ax[0].text(-122, 55, 'C. Rockies', color=color, rotation=0, fontsize=fontsize-1, fontweight=fontweight)
ax[0].text(-117.7, 47, 'S. Rockies', color=color, rotation=0, fontsize=fontsize-1, fontweight=fontweight)
# Example site location
min_snow_cover_stats_site = min_snow_cover_stats.loc[min_snow_cover_stats['RGIId']==site_name]
ax[0].plot(min_snow_cover_stats_site['CenLon'], min_snow_cover_stats_site['CenLat'], '*', 
           markeredgecolor='k', markerfacecolor='#e7298a', markersize=15, linewidth=2)
    
# -----b) Sentinel-2 image
ax[1].imshow(np.dstack([im_xr.B4.data[0], im_xr.B3.data[0], im_xr.B2.data[0]]),
             extent=(np.min(im_xr.x.data)/1e3, np.max(im_xr.x.data)/1e3, 
                     np.min(im_xr.y.data)/1e3, np.max(im_xr.y.data)/1e3))
ax[1].set_xlabel('Easting [km]')
ax[1].set_ylabel('Northing [km]')

# -----c) Classified image
ax[2].imshow(im_classified.classified.data[0], cmap=cmap_classified, clim=(1,5),
             extent=(np.min(im_classified.x.data)/1e3, np.max(im_classified.x.data)/1e3,
                     np.min(im_classified.y.data)/1e3, np.max(im_classified.y.data)/1e3))
ax[2].plot(np.divide(aoi_utm.geometry[0].exterior.coords.xy[0], 1e3), np.divide(aoi_utm.geometry[0].exterior.coords.xy[1], 1e3),
           '-k', label='Glacier boundary')
ax[2].set_xlabel('Easting [km]')
# dummy points for legend
ax[2].plot(0, 0, 's', color=cmap_classified(0), label='Snow')
ax[2].plot(0, 0, 's', color=cmap_classified(1), label='Shadowed snow')
ax[2].plot(0, 0, 's', color=cmap_classified(2), label='Ice/firn')
ax[2].plot(0, 0, 's', color=cmap_classified(3), label='Rock/debris')
ax[2].plot(0, 0, 's', color=cmap_classified(4), label='Water')
ax[2].set_xlim(ax[1].get_xlim())
ax[2].set_ylim(ax[1].get_ylim())
handles, labels = ax[2].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', ncols=6, markerscale=2, frameon=False)

# Plot AOI
for axis in ax[1:]:
    axis.plot(np.divide(aoi_utm.geometry[0].exterior.coords.xy[0], 1e3), 
              np.divide(aoi_utm.geometry[0].exterior.coords.xy[1], 1e3),
              '-k', linewidth=1, label='Glacier boundary')
    axis.set_yticks(np.arange(7030, 7046, step=5))

# Add text labels
text_labels = ['c', 'a', 'b']
for i in range(0, len(ax)):
    ax[i].text((ax[i].get_xlim()[1] - ax[i].get_xlim()[0]) * 0.9 + ax[i].get_xlim()[0],
               (ax[i].get_ylim()[1] - ax[i].get_ylim()[0]) * 0.85 + ax[i].get_ylim()[0],
                text_labels[i], fontweight='bold', fontsize=fontsize+4, horizontalalignment='center',
              bbox=dict(facecolor='w', edgecolor='None', pad=3))

# # Add caption
# caption = (r"\noindent\textbf{Figure 1. a)} Sentinel-2 surface reflectance image captured 2023-08-02 for one glacier (Randolph Glacier Inventory ID = 1.00037) \\"
#             r"and the associated \textbf{b)} classified image generated from the automated snow detection pipeline. \textbf{c)} Map of the study glacier locations, \\"
#             r"with marker sizes indicating the median accumulation area ratio (AAR) for the 2013–2023 study period. The maroon start marks the \\"
#             r"location of the example glacier shown in panels \textbf{a} and \textbf{b}." )
# fig.text(0.05, -0.02, caption, ha='left', wrap=True, fontsize=fontsize+1)

fig.tight_layout()
plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'agu24_abstract_figure.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)


### Snow cover GIF

In [ ]:
import glob

# Load inputs
rgi_id = "RGI60-01.00312"
aoi_fn = os.path.join(scm_path, 'study-sites', rgi_id, 'AOIs', f"{rgi_id}_outline.shp")
aoi = gpd.read_file(aoi_fn)
im_classified_fns = sorted(glob.glob(os.path.join(scm_path, 'study-sites', rgi_id, 'imagery', 'classified', '*.nc')))
scs_fn = os.path.join(scm_path, 'study-sites', rgi_id, f"{rgi_id}_snow_cover_stats.csv")
scs = pd.read_csv(scs_fn)
scs['datetime'] = pd.to_datetime(scs['datetime'], format='mixed')
# subset to 2019
im_classified_fns = [x for x in im_classified_fns if int(os.path.basename(x)[0:4]) == 2019] 
scs = scs.loc[scs['datetime'].dt.year == 2019]
out_path = os.path.join(figures_out_path, 'timeseries_gif')
if not os.path.exists(out_path):
    os.mkdir(out_path)
    print('Made directory for outputs:', out_path)

# Define colormap for classified images
cmap_dict = {"Snow": "#4eb3d3",  "Shadowed_snow": "#636363", "Ice": "#084081", "Rock": "#fe9929", "Water": "#252525"}
colors = []
for key in list(cmap_dict.keys()):
    color = list(matplotlib.colors.to_rgb(cmap_dict[key]))
    if key=='Rock':
        color += [0.5]
    colors.append(color)
        
cmap = matplotlib.colors.ListedColormap(colors)
cmap

In [ ]:
### Download images for 2019 ###

import math
import ee
import geedim as gd
import datetime
from rasterio.features import geometry_mask

def convert_wgs_to_utm(lon: float, lat: float):
    utm_band = str((math.floor((lon + 180) / 6) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0' + utm_band
    if lat >= 0:
        epsg_code = 'EPSG:326' + utm_band
        return epsg_code
    epsg_code = 'EPSG:327' + utm_band
    return epsg_code
    
def query_gee_for_imagery_run_pipeline(dataset, aoi_utm, date_start, date_end,
                                       month_start, month_end, site_name, 
                                       mask_clouds=True, cloud_cover_max=70, aoi_coverage=70, im_out_path=None,
                                       verbose=False, im_download=False):

    # -----Grab optimal UTM zone from AOI CRS
    epsg_utm = str(aoi_utm.crs.to_epsg())

    # -----Reformat AOI for image filtering
    # reproject CRS from AOI to WGS
    aoi_wgs = aoi_utm.to_crs('EPSG:4326')
    # prepare AOI for querying geedim (AOI bounding box)
    region = {'type': 'Polygon',
              'coordinates': [[[aoi_wgs.geometry.bounds.minx[0], aoi_wgs.geometry.bounds.miny[0]],
                               [aoi_wgs.geometry.bounds.maxx[0], aoi_wgs.geometry.bounds.miny[0]],
                               [aoi_wgs.geometry.bounds.maxx[0], aoi_wgs.geometry.bounds.maxy[0]],
                               [aoi_wgs.geometry.bounds.minx[0], aoi_wgs.geometry.bounds.maxy[0]],
                               [aoi_wgs.geometry.bounds.minx[0], aoi_wgs.geometry.bounds.miny[0]]
                               ]]}

    # -----Define function to query GEE for imagery
    def query_gee(dataset, date_start, date_end, region, cloud_cover_max, mask_clouds):
        if dataset == 'Landsat8':
            # Landsat 8
            im_col_gd = gd.MaskedCollection.from_name('LANDSAT/LC08/C02/T1_L2').search(start_date=date_start,
                                                                                       end_date=date_end,
                                                                                       region=region,
                                                                                       cloudless_portion=100 - cloud_cover_max,
                                                                                       mask=mask_clouds)
        elif dataset == 'Landsat9':
            # Landsat 9
            im_col_gd = gd.MaskedCollection.from_name('LANDSAT/LC09/C02/T1_L2').search(start_date=date_start,
                                                                                       end_date=date_end,
                                                                                       region=region,
                                                                                       cloudless_portion=100 - cloud_cover_max,
                                                                                       mask=mask_clouds)
        elif dataset == 'Sentinel-2_TOA':
            im_col_gd = gd.MaskedCollection.from_name('COPERNICUS/S2_HARMONIZED').search(start_date=date_start,
                                                                                         end_date=date_end,
                                                                                         region=region,
                                                                                         cloudless_portion=100 - cloud_cover_max,
                                                                                         mask=mask_clouds)

        elif dataset == 'Sentinel-2_SR':
            im_col_gd = gd.MaskedCollection.from_name('COPERNICUS/S2_SR_HARMONIZED').search(start_date=date_start,
                                                                                            end_date=date_end,
                                                                                            region=region,
                                                                                            cloudless_portion=100 - cloud_cover_max,
                                                                                            mask=mask_clouds)
        else:
            print("'dataset' variable not recognized. Please set to 'Landsat', 'Sentinel-2_TOA', or 'Sentinel-2_SR'. "
                  "Exiting...")
            return 'N/A'

        return im_col_gd

    # -----Define function to filter image IDs by month range
    def filter_im_ids_month_range(im_ids, im_dts, month_start, month_end):
        i = [int(ii) for ii in np.arange(0, len(im_dts)) if
             (im_dts[ii].month >= month_start) and (im_dts[ii].month <= month_end)]  # indices of images to keep
        im_ids, im_dts = [im_ids[ii] for ii in i], [im_dts[ii] for ii in i]  # subset of image IDs and datetimes
        # return 'N/A' if no images remain after filtering by month range
        if len(im_dts) < 1:
            return 'N/A', 'N/A'
        return im_ids, im_dts

    # -----Define function to couple image IDs captured within the same hour for mosaicking
    def image_mosaic_ids(im_col_gd):
        # Grab image properties, IDs, and datetimes from image collection
        properties = im_col_gd.properties
        ims = dict(properties).keys()
        im_ids = [properties[im]['system:id'] for im in ims]
        # return if no images found
        if len(im_ids) < 1:
            return 'N/A', 'N/A'
        im_dts = np.array(
            [datetime.datetime.utcfromtimestamp(properties[im]['system:time_start'] / 1000) for im in ims])

        # Remove image datetimes and IDs outside the specified month range
        im_ids, im_dts = filter_im_ids_month_range(im_ids, im_dts, month_start, month_end)

        # Grab all unique hours in image datetimes
        hours = np.array(im_dts, dtype='datetime64[h]')
        unique_hours = sorted(set(hours))

        # Create list of IDs for each unique hour
        im_mosaic_ids_list, im_mosaic_dts_list = [], []
        for unique_hour in unique_hours:
            i = list(np.ravel(np.argwhere(hours == unique_hour)))
            im_ids_list_hour = [im_ids[ii] for ii in i]
            im_mosaic_ids_list.append(im_ids_list_hour)
            im_dts_list_hour = [im_dts[ii] for ii in i]
            im_mosaic_dts_list.append(im_dts_list_hour)

        return im_mosaic_ids_list, im_mosaic_dts_list

    # -----Define function for extracting valid image IDs
    def extract_valid_image_ids(ds, date_start, date_end, region, cloud_cover_max, mask_clouds):
        # Initialize list of date ranges for querying
        date_ranges = [(date_start, date_end)]
        # Initialize list of error dates
        error_dates = []
        # Initialize error flag
        error_occurred = True
        # Iterate until no errors occur
        while error_occurred:
            error_occurred = False  # Reset the error flag at the beginning of each iteration
            try:
                # Initialize list of image collections
                im_col_gd_list = []
                # Iterate over date ranges
                for date_range in date_ranges:
                    # Query GEE for imagery
                    im_col_gd = query_gee(ds, date_range[0], date_range[1], region, cloud_cover_max, mask_clouds)
                    properties = im_col_gd.properties  # Error will occur here if an image is inaccessible!
                    im_col_gd_list.append(im_col_gd)
                # Initialize list of filtered image IDs and datetimes
                im_mosaic_ids_list_full, im_mosaic_dts_list_full = [], []  # Initialize lists of
                # Filter image IDs for month range and couple IDs for mosaicking
                for im_col_gd in im_col_gd_list:
                    im_mosaic_ids_list, im_mosaic_dts_list = image_mosaic_ids(im_col_gd)
                    if type(im_mosaic_ids_list) is str:
                        return 'N/A', 'N/A'
                    # append to list
                    im_mosaic_ids_list_full = im_mosaic_ids_list_full + im_mosaic_ids_list
                    im_mosaic_dts_list_full = im_mosaic_dts_list_full + im_mosaic_dts_list

                return im_mosaic_ids_list_full, im_mosaic_dts_list_full

            except Exception as e:
                error_id = str(e).split('ID=')[1].split(')')[0]
                print(f"Error querying GEE for {str(error_id)}")

                # Parse the error date from the exception message (replace this with your actual parsing logic)
                error_date = datetime.datetime.strptime(error_id[0:8], '%Y%m%d')
                error_dates.append(error_date)

                # Update date ranges excluding the problematic date
                date_starts = [date_start] + [str(error_date + datetime.timedelta(days=1))[0:10] for error_date in
                                              error_dates]
                date_ends = [str(error_date - datetime.timedelta(days=1))[0:10] for error_date in error_dates] + [
                    date_end]
                date_ranges = list(zip(date_starts, date_ends))

                # Set the error flag to indicate that an error occurred
                error_occurred = True

    # -----Apply functions
    if dataset == 'Landsat':  # must run Landsat 8 and 9 separately
        im_ids_list_8, im_dts_list_8 = extract_valid_image_ids('Landsat8', date_start, date_end, region,
                                                               cloud_cover_max, mask_clouds)
        im_ids_list_9, im_dts_list_9 = extract_valid_image_ids('Landsat9', date_start, date_end, region,
                                                               cloud_cover_max, mask_clouds)
        if (type(im_ids_list_8) is str) and (type(im_ids_list_9) is str):
            im_ids_list, im_dts_list = 'N/A', 'N/A'
        elif type(im_ids_list_9) is str:
            im_ids_list, im_dts_list = im_ids_list_8, im_dts_list_8
        elif type(im_ids_list_8) is str:
            im_ids_list, im_dts_list = im_ids_list_9, im_dts_list_9
        else:
            im_ids_list = im_ids_list_8 + im_ids_list_9
            im_dts_list = im_dts_list_8 + im_dts_list_9
    else:
        im_ids_list, im_dts_list = extract_valid_image_ids(dataset, date_start, date_end, region, cloud_cover_max,
                                                           mask_clouds)

    # -----Check if any images were found after filtering
    if type(im_ids_list) is str:
        print('No images found or error in one or more image IDs, exiting...')
        return 'N/A'

    if dataset=='Landsat':
        res = 30
        image_scalar = 36363.63636363636
        no_data_value = 0
    elif 'Sentinel-2' in dataset:
        res = 10
        image_scalar = 1e4
        no_data_value = -9999

    # -----Create xarray.Datasets from list of image IDs
    # loop through image IDs
    for i in tqdm(range(0, len(im_ids_list))):

        # subset image IDs and image datetimes
        im_ids, im_dts = im_ids_list[i], im_dts_list[i]

        # make directory for outputs (out_path) if it doesn't exist
        if not os.path.exists(im_out_path):
            os.mkdir(im_out_path)
            print('Made directory for image downloads: ' + im_out_path)
            
        # define filename
        if len(im_dts) > 1:
            im_fn = dataset + '_' + str(im_dts[0]).replace('-', '')[0:8] + '_MOSAIC.tif'
        else:
            im_fn = dataset + '_' + str(im_dts[0]).replace('-', '')[0:8] + '.tif'
            
        # check file does not already exist in directory, download
        if not os.path.exists(os.path.join(im_out_path, im_fn)):
            # create list of MaskedImages from IDs
            im_gd_list = [gd.MaskedImage.from_id(im_id) for im_id in im_ids]
            # combine into new MaskedCollection
            im_collection = gd.MaskedCollection.from_list(im_gd_list)
            # create image composite
            im_composite = im_collection.composite(method=gd.CompositeMethod.q_mosaic,
                                                   mask=mask_clouds,
                                                   region=region)
            # download to file
            im_composite.download(os.path.join(im_out_path, im_fn),
                                  region=region,
                                  scale=res,
                                  crs='EPSG:4326',
                                  dtype='int16',
                                  bands=im_composite.refl_bands)

    return

# Reproject AOI to optimal UTM zone
epsg_utm = convert_wgs_to_utm(aoi.geometry[0].centroid.coords.xy[0][0], aoi.geometry[0].centroid.coords.xy[1][0])
aoi_utm = aoi.to_crs(epsg_utm)

ee.Initialize()

# Landsat
dataset = 'Landsat'
query_gee_for_imagery_run_pipeline(dataset, aoi_utm, "2019-01-01", "2019-12-01", 5, 11, rgi_id, 
                                   mask_clouds=True, cloud_cover_max=70, aoi_coverage=70, im_out_path=out_path,
                                   verbose=False, im_download=True)

# Sentinel-2 SR
dataset = 'Sentinel-2_SR'
query_gee_for_imagery_run_pipeline(dataset, aoi_utm, "2019-01-01", "2019-12-01", 5, 11, rgi_id, 
                                   mask_clouds=True, cloud_cover_max=70, aoi_coverage=70, im_out_path=out_path,
                                   verbose=False, im_download=True)

# Sentinel-2 SR
dataset = 'Sentinel-2_TOA'
query_gee_for_imagery_run_pipeline(dataset, aoi_utm, "2019-01-01", "2019-12-01", 5, 11, rgi_id, 
                                   mask_clouds=True, cloud_cover_max=70, aoi_coverage=70, im_out_path=out_path,
                                   verbose=False, im_download=True)

# Remove images with < 70% coverage of AOI
fns = sorted(glob.glob(os.path.join(out_path, '*.tif')))
fn_remove_list = []
for fn in tqdm(fns):
    im = rxr.open_rasterio(fn).squeeze()
    im = im.rio.reproject(epsg_utm)
    im = im.isel(band=0)
    mask = geometry_mask(aoi_utm.geometry, transform=im.rio.transform(), invert=True, out_shape=im.shape)
    im_masked = xr.where(mask==1, im, np.nan)
    im_masked = xr.where((im_masked==-9999) | (im_masked==-32768), np.nan, im_masked)
    nreal = im_masked.notnull().sum().item()
    npx = np.count_nonzero(mask)
    percentage_real = (nreal / npx) * 100
    if percentage_real < 70:
        fn_remove_list.append(fn)
# for fn in fn_remove_list:
#     os.remove(fn)

In [ ]:
# Iterate over classified images
for im_classified_fn in tqdm(im_classified_fns):
    # Load classified image
    im_classified = rxr.open_rasterio(im_classified_fn, decode_times=False).squeeze()
    im_classified = xr.where(im_classified==-9999, np.nan, im_classified)
    im_classified = xr.where(im_classified==2, 1, im_classified)
    date = pd.Timestamp(datetime.datetime.strptime(os.path.basename(im_classified_fn).split('_')[0], "%Y%m%dT%H%M%S"))
    source = os.path.basename(im_classified_fn).split(rgi_id + '_')[1].split('_classified.nc')[0]

    # Load multispec image
    try:
        im_fn = glob.glob(os.path.join(out_path, f"{source}_{str(date)[0:10].replace('-', '')}*.tif"))[0]
    except:
        continue
    im = rxr.open_rasterio(im_fn).squeeze()
    if source=='Landsat':
        image_scaler = 36363.63636363636
    else:
        image_scaler = 1e4
    im = xr.where((im==-9999) | (im==-32768), np.nan, im / image_scaler)
    
    # Plot
    fig = plt.figure(figsize=(8,8))
    gs = matplotlib.gridspec.GridSpec(2, 2, figure=fig, height_ratios=[2,1])
    ax = [fig.add_subplot(gs[0,0]), fig.add_subplot(gs[0,1]), fig.add_subplot(gs[1,:])]
    # RGB image
    ax[0].imshow(np.dstack([im.isel(band=2).data, im.isel(band=1).data, im.isel(band=0).data]),
                 extent=(np.min(im.x.data), np.max(im.x.data), np.min(im.y.data), np.max(im.y.data)))
    aoi.plot(ax=ax[0], facecolor='None', edgecolor='k')
    # classified image
    xmin, xmax = -145.3555, -145.045
    ymin, ymax = 63.144, 63.325
    ax[1].imshow(im_classified.data, cmap=cmap, clim=(1,5), 
                 extent=(xmin, xmax, ymax, ymin))
    ax[1].invert_yaxis()
    aoi.plot(ax=ax[1], facecolor='None', edgecolor='k')
    # dummy points for legend
    ax[1].plot(0, 0, 's', color=colors[0], markersize=12, label='Snow')
    ax[1].plot(0, 0, 's', color=colors[2], markersize=12, label='Ice')
    ax[1].plot(0, 0, 's', color=colors[3], markersize=12, label='Rock')
    ax[1].plot(0, 0, 's', color=colors[4], markersize=12, label='Water')
    ax[1].legend(loc='lower right', frameon=False)
    for axis in ax[0:2]:
        axis.set_xlim(xmin, xmax)
        axis.set_ylim(ymin, ymax)
        axis.set_xticks(axis.get_xticks()[1::2])
        axis.set_yticks(axis.get_yticks()[1::2])

    # AAR time series
    ax[2].plot(scs['datetime'], scs['AAR'], '.k')
    scs_date = scs.loc[(scs['datetime']==date) & (scs['source']==source)]
    ax[2].plot(scs_date['datetime'], scs_date['AAR'], '*m', markersize=15)
    ax[2].set_ylim(0,1.05)
    ax[2].grid(True)
    ax[2].set_ylabel('Snow area ratio')
    fig.suptitle(f"{date}\n{source.replace('_', ' ')}")
    
    fig.tight_layout()

    fig_fn = os.path.join(out_path, f"{date}_{rgi_id}_snow_cover.png")
    fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
    plt.close()
        
    

### Map of study sites with climate clusters

In [ ]:
# Load climate clusters / mean climate
clusters = pd.read_csv(clusters_fn)
print('Climate clusters loaded')

# Load AOIs
aois = gpd.read_file(aois_fn)
# Add climate cluster column
aois = aois.merge(clusters[['RGIId', 'clustName']], on='RGIId')
aois.rename(columns={'clustName': 'Climate class'}, inplace=True)
print('AOIs loaded')

# Load RGI O2 Regions
rgi_O2_fn = os.path.join(scm_path, '..', 'GIS_data', 'RGI', 'RGIv7_02Regions', 
                                'RGI2000-v7.0-o2regions-Alaska-westernCanadaUS_clipped_to_country_outlines.shp')
rgi_O2 = gpd.read_file(rgi_O2_fn)
# remove Brooks Range
rgi_O2 = rgi_O2.loc[rgi_O2['o2region']!='01-01']
# add subregion name and color column
rgi_O2[['Subregion', 'color']] = '', ''
for i, o1o2 in enumerate(rgi_O2['o2region'].drop_duplicates().values):
    o1 = int(o1o2[0:2])
    o2 = int(o1o2[3:])
    subregion_name, color = f.determine_subregion_name_color(o1, o2)
    rgi_O2.loc[rgi_O2['o2region']==o1o2, 'Subregion'] = subregion_name
print('RGI O2 regions loaded')

# Load GTOPO30
gtopo_fn = '/Users/raineyaberle/Research/PhD/GIS_data/GTOPO30_clip.tif'
gtopo = rxr.open_rasterio(gtopo_fn)
gtopo = xr.where(gtopo==-32768, np.nan, gtopo)
print('GTOPO30 loaded')


In [ ]:
# Set up figure
fontsize=12
plt.rcParams.update({'font.size':fontsize, 'font.sans-serif':'Arial'})
fig, ax = plt.subplots(figsize=(10,10))

# GTOPO hillshade
ls = matplotlib.colors.LightSource(azdeg=90, altdeg=45)
ax.imshow(ls.hillshade(gtopo.data[0], vert_exag=0.002), cmap='gray', alpha=0.5,
             extent=(np.min(gtopo.x.data), np.max(gtopo.x.data), 
                     np.min(gtopo.y.data), np.max(gtopo.y.data)))
# RGI O2 region outlines
color = '#525252'
rgi_O2.plot(ax=ax, alpha=1.0, facecolor='None', edgecolor=color, linewidth=1)
ax.set_yticks(np.linspace(45, 65, num=6))
ax.set_xlim(-167, -112)
ax.set_ylim(46, 66.5)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_aspect(2.1)
# Site locations
obj = sns.scatterplot(data=aois, x='CenLon', y='CenLat', edgecolor='k', linewidth=0.5, 
                      hue='Climate class', hue_order=cluster_order, palette=cluster_cmap_dict, alpha=1, ax=ax)
sns.move_legend(obj, loc='lower left', markerscale=2, bbox_to_anchor=[0.15, 0.1, 0.2, 0.2])
# Add region labels and arrows
fontweight = 'bold'
ax.text(-163, 56, 'Aleutians', color=color, rotation=35, fontsize=fontsize-1, fontweight=fontweight)
ax.text(-158, 62.3, 'Alaska Range', color=color, rotation=0, fontsize=fontsize-1, fontweight=fontweight)
ax.text(-147.9, 57.8, 'W. Chugach \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-1, fontweight=fontweight)
ax.arrow(-147.6, 58.8, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax.text(-141.7, 57.7, 'St. Elias \nMtns.', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-1, fontweight=fontweight)
ax.arrow(-141.5, 58.7, 0, 0.8, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax.text(-139.6, 56.4, 'N. Coast \nRanges', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-1, fontweight=fontweight)
ax.arrow(-137.3, 56.8, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax.text(-133, 51.3, 'N. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-1, fontweight=fontweight)
ax.arrow(-129.4, 51.4, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax.text(-129.7, 47, 'S. Cascades', color=color, rotation=0, horizontalalignment='center', fontsize=fontsize-1, fontweight=fontweight)
ax.arrow(-126, 47.1, 1.3, 0, color=color, linewidth=2, head_width=0.25, head_length=0.2)
ax.text(-132, 64, 'N. Rockies', color=color, rotation=0, fontsize=fontsize-1, fontweight=fontweight)
ax.text(-122, 55, 'C. Rockies', color=color, rotation=0, fontsize=fontsize-1, fontweight=fontweight)
ax.text(-117.7, 47, 'S. Rockies', color=color, rotation=0, fontsize=fontsize-1, fontweight=fontweight)

plt.show()

# Save figure
fig_fn = os.path.join(figures_out_path, 'agu24_study_sites_map.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)


### Model SLA animation

In [ ]:
import glob
plt.rcParams.update({'font.size':12, 'font.sans-serif': 'Arial'})

# Define output directory
out_path = os.path.join(figures_out_path, 'model_SMB_to_SLA_animation')
if not os.path.exists(out_path):
    os.mkdir(out_path)
    print('Made directory for outputs:', out_path)
    
# Grab modeled SMB file names
rgi_id = '1.00032'
bin_fn = sorted(glob.glob(os.path.join(scm_path, 'Rounce_et_al_2023', 'binned', f'{rgi_id}*.nc')))[0]

# Load binned data
bin = xr.open_dataset(bin_fn)
# grab data variables
h = bin.bin_surface_h_initial.data[0] # surface elevation [m]
x = bin.bin_distance.data[0]
b_sum = np.zeros((len(bin.time.data), len(h))) # cumulative SMB
times = [np.datetime64(x) for x in bin.time.data] # datetimes
months = list(pd.DatetimeIndex(times).month) # months of each datetime
# iterate over each time period after 2013
times = [time for time in times if time >= np.datetime64('2012-10-01')]
elas = np.nan*np.zeros(len(times)) # initialize transient ELAs
for j, time in enumerate(tqdm(times)):
    # subset binned data to time
    bin_time = bin.isel(time=j)
    # grab the SMB 
    b_sum[j,:] = bin_time.bin_massbalclim_monthly.data[0]
    # add the previous SMB (restart the count in October)
    if months[j] != 10: 
        b_sum[j,:] += b_sum[j-1,:]
    # If all SMB > 0, ELA = minimum elevation
    if all(b_sum[j,:] > 0):
        elas[j] = np.min(h)
    # If SMB is > 0 and < 0 in some places, linearly interpolate ELA
    elif any(b_sum[j,:] < 0) & any(b_sum[j,:] > 0):
        elas[j] = np.interp(0, np.flip(b_sum[j,:]), np.flip(h))
    # If SMB < 0 everywhere, fit a piecewise linear fit and extrapolate for SMB=0
    elif all(b_sum[j,:] < 0):
        X, y = b_sum[j,:], h
        elas[j] = np.nanmax(h)
    else:
        print('issue')
        
    # Plot results
    if time >= np.datetime64('2013-01-01'):
        fig, ax = plt.subplots(2, 1, gridspec_kw=dict(height_ratios=[3,1]), figsize=(6,6))
        # surface profile
        ax[0].fill_between(x, np.zeros(len(x)), h, color='gray', edgecolor='k', alpha=0.5)
        positive_mask = b_sum[j,:] > 0
        negative_mask = b_sum[j,:] < 0
        # positive mass balance bars
        ax[0].bar(x[positive_mask], b_sum[j,positive_mask]*50, width=126, 
            bottom=h[positive_mask], color='blue', alpha=0.6, label='Positive Mass Balance')
        # negative mass balance bars
        ax[0].bar(x[negative_mask], b_sum[j,negative_mask]*50, width=126, 
                bottom=h[negative_mask], color='red', alpha=0.6, label='Negative Mass Balance')
        # SLA
        ax[0].axhline(elas[j], 0, np.nanmax(x), color='k')
        ax[0].text(7e3, elas[j]+50, 'Snowline altitude', color='k', ha='right')
        ax[0].set_ylim(1.25e3, 3.3e3)
        ax[0].set_yticks([1500, 2000, 2500, 3000])
        ax[0].set_ylabel('Elevation [m]')
        ax[0].set_xlim(0, np.nanmax(x))
        ax[0].set_xticks(np.arange(0, 7.1e3, step=1e3))
        ax[0].set_xticklabels(np.divide(ax[0].get_xticks(), 1e3).astype(int).astype(str))
        ax[0].set_xlabel('km')
        ax[0].spines[['top', 'right']].set_visible(False)
        ax[0].set_title(str(time)[0:7])
        # SLA time series
        ax[1].plot(times, elas, '.k', markersize=5)
        ax[1].plot(times[j], elas[j], '*m', markersize=10)
        ax[1].set_xlim(np.datetime64('2013-01-01'), np.datetime64('2023-01-01'))
        ax[1].set_ylim(1330, 2600)
        ax[1].set_ylabel('Snowline altitude [m]')
        fig.tight_layout()
        # Save figure
        fig_fn = os.path.join(out_path, f"{str(time)[0:7]}.png")
        fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
        plt.close()
        

In [ ]:
## Site location map

# # Load GTOPO30
# gtopo_fn = '/Users/raineyaberle/Research/PhD/GIS_data/GTOPO30_clip.tif'
# gtopo = rxr.open_rasterio(gtopo_fn)
# gtopo = xr.where(gtopo==-32768, np.nan, gtopo)
# print('GTOPO30 loaded')
# Load country outlines
countries_fn = '/Users/raineyaberle/Research/PhD/GIS_data/countries_shp/countries.shp'
countries = gpd.read_file(countries_fn)
countries = countries.loc[(countries['NAME']=='Canada') | (countries['NAME']=='United States')]

# Set up figure
fontsize=12
plt.rcParams.update({'font.size':fontsize, 'font.sans-serif':'Arial'})
fig, ax = plt.subplots(figsize=(8,8))
# GTOPO hillshade
ls = matplotlib.colors.LightSource(azdeg=90, altdeg=45)
ax.imshow(ls.hillshade(gtopo.data[0], vert_exag=0.002), cmap='gray', alpha=0.5,
             extent=(np.min(gtopo.x.data), np.max(gtopo.x.data), 
                     np.min(gtopo.y.data), np.max(gtopo.y.data)))
# country outlines
countries.plot(ax=ax, facecolor='None', edgecolor='gray')
# Glaciers
# aois.plot(facecolor='#08519c', edgecolor='None', ax=ax)
# Site location
rgi_id = 'RGI60-01.00032'
site_centroid = aois.loc[aois['RGIId']==rgi_id].geometry[0].centroid.coords.xy
ax.plot(*site_centroid, '*', markersize=40, markerfacecolor='m', markeredgecolor='k', linewidth=0.5)

ax.set_xlim(-170, -120)
ax.set_ylim(45, 71)
ax.set_aspect(2.1)
ax.set_axis_off()

# Save figure
fig_fn = os.path.join(figures_out_path, 'model_SMB_to_SLA_animation', 'site_location_map.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)